In [23]:
from glob import glob
import pandas as pd
import numpy as np
import os
from sklearn.utils import shuffle
from keras.utils import to_categorical

In [24]:
path = glob('/Users/chief.andreev2017yandex.ru/Desktop/cats_dogs_dataset/*.txt')

In [50]:
#разделение файлов по папкам
for x in path:
    with open(x,'r') as f:
        arr = np.array(list(map(int,f.readline().split())))
        my_path = x.split('/')
        if arr[0]==1:
            os.replace(x, "/Users/chief.andreev2017yandex.ru/Desktop/cats/{}".format(my_path[-1]))
            os.replace(x[0:-4]+'.jpg',"/Users/chief.andreev2017yandex.ru/Desktop/cats/{}".format(my_path[-1][0:-4]+'.jpg'))
        else:
            os.replace(x, "/Users/chief.andreev2017yandex.ru/Desktop/dogs/{}".format(my_path[-1]))
            os.replace(x[0:-4]+'.jpg',"/Users/chief.andreev2017yandex.ru/Desktop/dogs/{}".format(my_path[-1][0:-4]+'.jpg'))

In [25]:
path = glob('/Users/chief.andreev2017yandex.ru/Desktop/cats/*.txt')
aim = []
xmin = []
xmax = []
ymin = []
ymax = []
columns = ['aim','xmin','ymin','xmax','ymax'] 
for x in path:
    with open(x,'r') as f:
        arr = np.array(list(map(int,f.readline().split())))
        aim.append(arr[0])
        xmin.append(arr[1])
        ymin.append(arr[2])
        xmax.append(arr[3])
        ymax.append(arr[4])
d = {'path':x[0:-4]+'.jpg','xmin':xmin,'ymin':ymin,'xmax':xmax,'ymax':ymax,'label':aim}
cats = pd.DataFrame(data=d)

In [26]:
path = glob('/Users/chief.andreev2017yandex.ru/Desktop/dogs/*.txt')
aim = []
xmin = []
xmax = []
ymin = []
ymax = []
columns = ['aim','xmin','ymin','xmax','ymax'] 
for x in path:
    with open(x,'r') as f:
        arr = np.array(list(map(int,f.readline().split())))
        aim.append(arr[0])
        xmin.append(arr[1])
        ymin.append(arr[2])
        xmax.append(arr[3])
        ymax.append(arr[4])
d = {'path':x[0:-4]+'.jpg','xmin':xmin,'ymin':ymin,'xmax':xmax,'ymax':ymax,'label':aim}
dogs = pd.DataFrame(data=d)

In [27]:
dogs.to_csv('dogs.csv')
cats.to_csv('cats.csv')

Мы разделили картинки на 2 класса. Созданы 2 DataFrame для последующей проверки(Порядок строк такой же, как порядок файлов)

In [28]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import os

In [29]:
data = []
labels = []
bboxes = []
imagePaths = []

In [30]:
cats

,path,xmin,ymin,xmax,ymax,label
0,/Users/chief.andreev2017yandex.ru/Desktop/cats...,39,51,156,179,1
1,/Users/chief.andreev2017yandex.ru/Desktop/cats...,111,47,240,154,1
2,/Users/chief.andreev2017yandex.ru/Desktop/cats...,17,25,206,180,1
3,/Users/chief.andreev2017yandex.ru/Desktop/cats...,215,23,393,176,1
4,/Users/chief.andreev2017yandex.ru/Desktop/cats...,138,107,259,227,1
...,...,...,...,...,...,...
1032,/Users/chief.andreev2017yandex.ru/Desktop/cats...,46,22,166,168,1
1033,/Users/chief.andreev2017yandex.ru/Desktop/cats...,204,145,286,231,1
1034,/Users/chief.andreev2017yandex.ru/Desktop/cats...,151,71,335,267,1
1035,/Users/chief.andreev2017yandex.ru/Desktop/cats...,222,64,351,185,1


In [31]:
cats['path'][0]

'/Users/chief.andreev2017yandex.ru/Desktop/cats/Bengal_184.jpg'

In [32]:
dogs['path']=glob('/Users/chief.andreev2017yandex.ru/Desktop/dogs/*.jpg')
cats['path']=glob('/Users/chief.andreev2017yandex.ru/Desktop/cats/*.jpg')

In [34]:
cats = shuffle(cats).reset_index(drop=True)
dogs = shuffle(dogs).reset_index(drop=True)


,path,xmin,ymin,xmax,ymax,label
0,/Users/chief.andreev2017yandex.ru/Desktop/cats...,97,60,280,234,1
1,/Users/chief.andreev2017yandex.ru/Desktop/cats...,162,9,217,64,1
2,/Users/chief.andreev2017yandex.ru/Desktop/cats...,152,17,381,262,1
3,/Users/chief.andreev2017yandex.ru/Desktop/cats...,135,66,268,193,1
4,/Users/chief.andreev2017yandex.ru/Desktop/cats...,53,16,243,196,1
...,...,...,...,...,...,...
1032,/Users/chief.andreev2017yandex.ru/Desktop/cats...,80,47,368,290,1
1033,/Users/chief.andreev2017yandex.ru/Desktop/cats...,96,35,234,185,1
1034,/Users/chief.andreev2017yandex.ru/Desktop/cats...,223,1,436,246,1
1035,/Users/chief.andreev2017yandex.ru/Desktop/cats...,134,58,221,139,1


In [35]:
%%time
data = []
labels = []
bboxes = []
imagePaths = []
for index, row in cats.iterrows():
    image = cv2.imread(row['path'])
    (h, w) = image.shape[:2]
    startX = float(row['xmin']) / w
    startY = float(row['ymin']) / h
    endX = float(row['xmax']) / w
    endY = float(row['ymax']) / h
    image = load_img(row['path'], target_size=(224, 224))
    image = img_to_array(image)
    data.append(image)
    labels.append(row['label'])
    bboxes.append((row['xmin'], row['ymin'], row['xmax'], row['ymax']))
    imagePaths.append(row['path'])

CPU times: user 12.1 s, sys: 1.63 s, total: 13.7 s
Wall time: 17.2 s


In [36]:
%%time
for index, row in dogs.iterrows():
    image = cv2.imread(row['path'])
    (h, w) = image.shape[:2]
    startX = float(row['xmin']) / w
    startY = float(row['ymin']) / h
    endX = float(row['xmax']) / w
    endY = float(row['ymax']) / h
    image = load_img(row['path'], target_size=(224, 224))
    image = img_to_array(image)
    data.append(image)
    labels.append(row['label'])
    bboxes.append((row['xmin'], row['ymin'], row['xmax'], row['ymax']))
    imagePaths.append(row['path'])

CPU times: user 25.2 s, sys: 3.48 s, total: 28.7 s
Wall time: 37.3 s


In [37]:
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
labels = to_categorical(labels)
bboxes = np.array(bboxes, dtype="float32")
imagePaths = np.array(imagePaths)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
if len(lb.classes_) == 2:
    labels = to_categorical(labels)

In [39]:
labels

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [40]:
split = train_test_split(data, labels, bboxes, imagePaths,test_size=0.20, random_state=42)
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]
(trainPaths, testPaths) = split[6:]

In [41]:
vgg = VGG16(weights="imagenet", include_top=False,
input_tensor=Input(shape=(224, 224, 3)))
vgg.trainable = False
flatten = vgg.output
flatten = Flatten()(flatten)

In [42]:
#Bounding box
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid",
name="bounding_box")(bboxHead)
# Метка класса
softmaxHead = Dense(512, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(3, activation="softmax",
name="class_label")(softmaxHead)
model = Model(inputs=vgg.input,outputs=(bboxHead, softmaxHead))

In [43]:
INIT_LR = 1e-4
NUM_EPOCHS = 11
BATCH_SIZE = 32
losses = {"class_label": "categorical_crossentropy","bounding_box": "mean_squared_error",}
lossWeights = {"class_label": 1.0,"bounding_box": 1.0}
opt = Adam(lr=INIT_LR)
model.compile(loss=losses, optimizer=opt, metrics=["accuracy"], loss_weights=lossWeights)
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

In [44]:
trainTargets = {"class_label": trainLabels,"bounding_box": trainBBoxes}
testTargets = {"class_label": testLabels,"bounding_box": testBBoxes}

In [45]:
H = model.fit(trainImages, trainTargets,validation_data=(testImages, testTargets),batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,verbose=1)


Train on 2708 samples, validate on 677 samples
Epoch 1/11
2708/2708 [==============================] - 906s 335ms/sample - loss: 47602.5517 - bounding_box_loss: 47575.9922 - class_label_loss: 0.5308 - bounding_box_acc: 0.8113 - class_label_acc: 0.7640 - val_loss: 44878.9704 - val_bounding_box_loss: 44827.8555 - val_class_label_loss: 0.3194 - val_bounding_box_acc: 0.8168 - val_class_label_acc: 0.8759
Epoch 2/11
2708/2708 [==============================] - 880s 325ms/sample - loss: 47596.6977 - bounding_box_loss: 47566.4141 - class_label_loss: 0.2934 - bounding_box_acc: 0.7962 - class_label_acc: 0.8807 - val_loss: 44878.9034 - val_bounding_box_loss: 44752.8125 - val_class_label_loss: 0.2729 - val_bounding_box_acc: 0.7903 - val_class_label_acc: 0.8715
Epoch 3/11
2708/2708 [==============================] - 882s 326ms/sample - loss: 47596.5928 - bounding_box_loss: 47587.3516 - class_label_loss: 0.2159 - bounding_box_acc: 0.7323 - class_label_acc: 0.9099 - val_loss: 44878.8320 - val_boundin

In [46]:
MODEL_PATH = os.path.sep.join(['/Users/chief.andreev2017yandex.ru/Documents/neurus_output/', "detector.h5"])
LB_PATH = os.path.sep.join(['/Users/chief.andreev2017yandex.ru/Documents/neurus_output/', "lb.pickle"])
PLOTS_PATH = os.path.sep.join(['/Users/chief.andreev2017yandex.ru/Documents/neurus_output/', "plots"])
TEST_PATHS = os.path.sep.join(['/Users/chief.andreev2017yandex.ru/Documents/neurus_output/', "test_paths.txt"])
print("Saving object detector model...")
model.save(MODEL_PATH, save_format="h5")
print("Saving label binarizer...")
f = open(LB_PATH, "wb")
f.write(pickle.dumps(lb))
f.close()

Saving object detector model...
Saving label binarizer...
